In [ ]:
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import scipy.stats
import scipy.optimize
from collections import defaultdict

In [ ]:
sdata = pd.read_csv('song_data.csv').set_index('song_name')
sinfo = pd.read_csv('song_info.csv').set_index('song_name')

In [ ]:
sdata.shape,sinfo.shape

In [ ]:
sdf = pd.merge(sdata,sinfo,on='song_name')
sdf['countD'] = 1
adf = sdf.groupby('artist_name').mean()
adfs = sdf.groupby('artist_name').sum()

In [ ]:
_ = plt.hist(adfs.countD,500,density=True)

In [ ]:
adf.sort_values('song_popularity',0,False)

In [ ]:
df = pd.read_csv('avg_stats.csv')#pd.read_csv('BBGM_League_7_all_seasons_Average_Stats.csv')
df = df[(df.G*df.MP > 48)]
df.shape
#df = df.sample(10000)
df.columns

In [ ]:
y = df.iloc[:,-15:]
X = df.iloc[:,11:-17]
#y = y[(X['AST%'] >0) & (X['AST%'] < 100)]
print(X.shape)
#X = X[(X['AST%'] >0) & (X['AST%'] < 100)]
print(X.shape)

X['MP'] = df.MP
X['Hgt'] = df['Hgt']

stat_list = ['FG','FGA','3P',"3PA",'FT','FTA',\
             'ORB','DRB','TRB','AST','TOV','STL',"Blk",\
             'PF','PTS']
for name in stat_list:
    den = np.maximum(1,df.MP)
    X[name + 'p36'] = 36* df[name]/den
    X[name + 'p100'] = X[name + 'p36']*4/3


X['PTSp36'] -= X['3Pp36']

#else:
#X['PER'] = df['PER']
    #X['DREO'] = X['PTSp100'] + .2*X['TRBp100'] + .5*X['ASTp100'] - .9*X['FGAp100'] - .35*X['FTAp100']-6.23
#X['PassP'] = ((X['ASTp100']-(0.38*X['Creation']))*0.752+ X['Creation'] + X['TOVp100']) ** 0.67
#'OPM','DPM','cTOV','Load'#stat_list[:-2]+
#'PER','FG%','DREO'



X = X[[_ for _ in X.columns if '%A' in _ or _[-1]=='r' or "+/-" in _ or 'p36' in _ or _ in (['PER','OSPM','DRE','OPM','BPM','DPM','Creation','cTOV','Load','Age','MP'])]]
if True:
    X = X[[_ for _ in X.columns if not '3P' in _]]
    X = X[[_ for _ in X.columns if not 'Rim' in _]]
    X = X[[_ for _ in X.columns if not 'Post' in _]]
    X = X[[_ for _ in X.columns if not 'Mid' in _]]
    X = X[[_ for _ in X.columns if not 'TOV' in _]]
    X = X[[_ for _ in X.columns if not 'Blk' in _]]
    X = X[[_ for _ in X.columns if not 'ORB' in _]]
    X = X[[_ for _ in X.columns if not 'DRB' in _]]
    X = X[[_ for _ in X.columns if not 'STL' in _]]
    X = X[[_ for _ in X.columns if not 'DPM' in _]]
    X = X[[_ for _ in X.columns if not 'OPM' in _]]
    X = X[[_ for _ in X.columns if not 'BPM' in _]]
    X = X[[_ for _ in X.columns if not '+/-' in _]]


#X = X[[_ for _ in X.columns if not '3PAr' in _]]

replacement_filter = (df.Salary > 0.5) & (df.Salary < 1.0)
replacement_player_mean_bs = X[replacement_filter].mean()
replacement_player_std_bs = X[replacement_filter].std()
replacement_player_cov_bs = X[replacement_filter].cov()

replacement_player_mean_r = y[replacement_filter].mean()
replacement_player_std_r = y[replacement_filter].std()
replacement_player_cov_r = y[replacement_filter].cov()


replacement_player_mean_r

X.columns


In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection, model_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm
fexp = preprocessing.PolynomialFeatures(degree=2,interaction_only=False)
scalerX = preprocessing.RobustScaler()
scalery = preprocessing.StandardScaler()
prescale_X = scalerX.fit_transform(X)
prescale_y = scalery.fit_transform(y)
prescale_X = fexp.fit_transform(prescale_X)
X_train, X_test, y_train, y_test = model_selection.train_test_split(prescale_X, prescale_y, test_size=0.25, random_state=42)
trials = 1
ts = []
for i in range(trials):
    #clf = neural_network.MLPRegressor((36,5,24,36),'tanh',solver='adam',max_iter=1000)
    #clf = neural_network.MLPRegressor((),'identity',solver='lbfgs',alpha=5e2,tol=1e-9)
    #clf = multioutput.MultiOutputRegressor(linear_model.SGDRegressor(penalty='l2',alpha=5e2,eta0=1e-6,tol=1e-12,max_iter=50,verbose=True))
    clf = multioutput.MultiOutputRegressor(linear_model.ElasticNet(alpha=5e-3,l1_ratio=0.5))
    #clf = ensemble.ExtraTreesRegressor(8,criterion='mae',max_depth=3,verbose=1)
    #clf = multioutput.MultiOutputRegressor(svm.SVR())
    #clf.fit(X_train,y_train)
    #print(clf.score(X_train,y_train),clf.score(X_test,y_test))
    clf.fit(prescale_X,prescale_y)
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    err = np.linalg.norm(yt-y)
    ts.append((err,clf))
ts = sorted(ts)[::1] # why not the biggest error
print(ts[0][0])
clf = ts[0][1]

In [ ]:
y_keys = [_.lower() for _ in y.columns]

y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }

In [ ]:
adf.columns

In [ ]:
stat_lookup = {'song_duration_ms': 'MP',
 'song_popularity': 'PTSp36',
 'acousticness': 'FTAp36',
 'energy': 'FGp36',
 'danceability': 'FTp36',
 'instrumentalness': 'TOVp36',
 'liveness': 'TRBp36',
 'speechiness': 'ASTp36',
 'loudness': 'PFp36',
 'tempo': 'FGAp36',
 'audio_valence': 'FTr'}

inv_lookup = {v: list(adf.columns).index(k) for k,v in stat_lookup.items()}
# 35
inv_lookup = {'FTr': 5,
 'MP': 0,
 'FGp36': 10,
 'FGAp36': 1,
 'FTp36': 12,
 'FTAp36': 9,
 'TRBp36': 11,
 'ASTp36': 3,
 'PFp36': 7,
 'PTSp36': 14}
# spearman
inv_lookup = {'FTr': 11,
 'MP': 0,
 'FGp36': 1,
 'FGAp36': 6,
 'FTp36': 14,
 'FTAp36': 4,
 'TRBp36': 10,
 'ASTp36': 9,
 'PFp36': 12,
 'PTSp36': 3}
# inv_lookup PER spearman
inv_lookup = {'PER': 0,
 'FTr': 10,
 'MP': 14,
 'FGp36': 11,
 'FGAp36': 12,
 'FTp36': 8,
 'FTAp36': 6,
 'TRBp36': 3,
 'ASTp36': 7,
 'PFp36': 4,
 'PTSp36': 2}
# per pearson
inv_lookup = {'PER': 0,
 'FTr': 11,
 'MP': 12,
 'FGp36': 9,
 'FGAp36': 6,
 'FTp36': 10,
 'FTAp36': 8,
 'TRBp36': 4,
 'ASTp36': 1,
 'PFp36': 14,
 'PTSp36': 2}

In [ ]:
player_vectors = []
player_names = []
for i,row in enumerate(adf.itertuples()):
    if adfs.countD[i] < 5:
        continue
    player_vectors.append([row[inv_lookup[stat]+1] for stat in X.columns])
    player_names.append(row[0])


In [ ]:
import random
Xn = np.nan_to_num(np.array(player_vectors))
scalerX2 = preprocessing.RobustScaler(quantile_range=(35.0, 65.0))
scalerX2.fit(Xn)
Xn_s =scalerX2.transform(np.nan_to_num(Xn))
Xn_fs = fexp.transform(np.nan_to_num(Xn_s))
predict = clf.predict(Xn_fs)
ratings = np.nan_to_num(scalery.inverse_transform(predict))
nc2 = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])/38
ovr = np.clip(ratings,0,100) @ nc2
#YEAR = np.clip(ratings,0,100) @ np.

top_n = 30*15
top_n

In [ ]:
is_active = np.zeros(ratings.shape[0])
is_active[:top_n] = 1
teamID = sum([[i]*15 for i in range(30)],[])
random.shuffle(teamID)
random.shuffle(is_active)
len(teamID)

In [ ]:
import random
players = []
tp = 0
for name,rating,ia in zip(player_names,ratings,is_active):
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])
    
    r_vec = {k: rating[y_keys.index(km)] for k,km in y_map.items()}
    r_vec = {k: int(np.clip(v,0,100)) for k,v in r_vec.items()}

    new_player['ratings'] = [r_vec]
    new_player['born'] = {'year':2019-25,'loc':''}

    if ia == 1:
        new_player['tid'] = teamID[tp]
        tp += 1
    else:
        new_player['tid'] = -2
        year = np.random.randint(1,11)
        new_player['born']['year'] += year
        new_player['draft'] = {'year':2018+year,"round": 0, "pick": 0, "tid": -1, "originalTid": -1}
    players.append(new_player)



In [ ]:
len(player_names),len(players)

In [ ]:
base = json.load(open('league_blank.json','rb'))
base['version'] = 33
del base['meta']

In [ ]:
base['players'] = players

In [ ]:
with open('music_{}.json'.format('spotifyAUTO'),'wt') as fp:
    json.dump(base,fp, sort_keys=True)

In [ ]:
adf[adfs.countD >5].sort_values('song_popularity',0,False)

In [ ]:
adfs.sort_values('song_popularity',0,False)

In [ ]:
def map_obj(x):
    x_ord = np.argsort(x)
    inv_lookup = {v: _ for v,_ in zip(X.columns,x_ord[:len(stat_lookup)])}
    player_vectors = []
    player_names = []
    for i,row in enumerate(adf.itertuples()):
        if adfs.countD[i] < 5:
            continue
        player_vectors.append([row[inv_lookup[stat]+1] for stat in X.columns])
        player_names.append(row[0])
    Xn = np.nan_to_num(np.array(player_vectors))
    scalerX2 = preprocessing.RobustScaler()#quantile_range=(30.0, 70.0))
    scalerX2.fit(Xn)
    Xn_s =scalerX2.transform(np.nan_to_num(Xn))
    Xn_fs = fexp.transform(np.nan_to_num(Xn_s))
    predict = clf.predict(Xn_fs)
    ratings = np.nan_to_num(scalery.inverse_transform(predict))
    nc2 = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])/38
    ovr = np.clip(ratings,0,100) @ nc2
    return -scipy.stats.pearsonr(ovr,adfs[adfs.countD >= 5]['song_popularity'])[0]
    

In [ ]:
#res = scipy.optimize.differential_evolution(map_obj,[(0,1) for i in adf.columns],disp=True)

In [ ]:
best_x = np.random.rand(len(adf.columns))
best_v = map_obj(best_x)
for i in range(1000):
    x = np.random.rand(len(adf.columns))
    v = map_obj(x)
    if v < best_v:
        best_x = x
        best_v = v
        print(best_v)

In [ ]:
x_ord = np.argsort(best_x)
inv_lookup = {v: _ for v,_ in zip(X.columns,x_ord[:len(stat_lookup)])}
inv_lookup

In [ ]:
inv_lookup

In [ ]:
map_obj(best_x)